# QAOA Demo

## Installation commands for packages outside the standard library:
        pip install qiskit
        pip install qiskit-aer

Importing packages...

In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import networkx as nx
import numpy as np
from scipy.optimize import minimize
from matplotlib import pyplot as plt
from helper_functions import *
np.random.seed(7)

In this demo, we will implement the quantum approximate optimization algorithm (QAOA) for solving Max-Cut problems. To begin, we will set up our graph and briefly discuss Max-Cut.

In [ ]:
edges = [(0, 1), (1, 2), (2, 3), (3, 0)]
weights = [1, 2, 1, 3]
# edges = [(0, 1), (1, 2), (2, 3), (3, 1), (3, 0)]
# weights = [1, 2, 8, 22, 11]

graph = nx.Graph()
for edge, weight in zip(edges, weights):
    graph.add_edge(edge[0], edge[1], weight=weight)

We can now make a plot to visualize our graph.

In [ ]:
pos = nx.shell_layout(graph)
nx.draw(graph, pos=pos, node_size=1000, with_labels=True, font_color='white', font_size=20)
graph
edge_labels = nx.get_edge_attributes(graph, "weight")
nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, font_size=16)
plt.show()

Using a function from helper_functions.py, we can calculate the exact maximum cut for our graph, which corresponds to a bipartition on the graph nodes that maximizes the total weight of the edges between nodes in separate partitions.

In [ ]:
partitioning, cut_value = naive_exact_maxcut(graph)
print('max-cut partitioning:', partitioning, '\nmaximum cut value:', cut_value)

Now, we will construct a QAOA circuit to solve the Max-Cut problem. To begin, we will define two subcircuits corresponding to our cost and mixer layers.

In [ ]:
n_qubits = len(graph.nodes())

def add_mixer_layer(circ, beta):
    for i in range(n_qubits):
        circ.rx(2*beta, i)

def add_cost_layer(circ, alpha):
    weights = nx.get_edge_attributes(graph, "weight")
    for edge in list(graph.edges()):
        control = edge[0]
        target = edge[1]
        circ.cx(control, target)
        circ.rz(alpha*weights[edge], target)
        circ.cx(control, target)

We now define a function to construct and return our QAOA circuit by repeatedly calling the cost and mixer subcircuit functions for each layer. Note that this construct circuit function admits parameters $\alpha_j$ and $\beta_j$ for each layer, which will be optimized variationally in the next step.

Before adding cost or mixer layers, we place a layer of hadamard gates. This serves to initialize the state into the equal superposition state, where every possible bitstring/cut has an equal probability of being measured.

In [ ]:
def construct_circuit(alphas, betas):
    n_layers = len(alphas)

    circ = QuantumCircuit(n_qubits)

    for i in range(n_qubits):
        circ.h(i)

    for i in range(n_layers):
        circ.barrier()
        add_cost_layer(circ, alphas[i])
        circ.barrier()
        add_mixer_layer(circ, betas[i])

    circ.measure_all()
    return circ

Let's build and draw the circuit now using only a sinlge layer to get a sense of the structure we're working with.

In [ ]:
circ = construct_circuit([1, 1], [1, 1])
circ.draw('mpl')

Now we have to set up our optimization loop. We start by building a function which, given a bitstring $x$ corresponding to a bipartition (a cut) on our graph, gives the value of that cut.

In [ ]:
def get_cut_value(x):
    A = nx.adjacency_matrix(graph).toarray()
    mu = -np.ones(n_qubits)@A
    sigma = A + np.diag(mu)
    x_arr = np.array([float(entry) for entry in x])
    return x_arr@sigma@x_arr

Next, we introduce a subroutine that will execute our QAOA circuit for a given set of $\alpha$ and $\beta$ parameters with a predetermined number of shots and give the average cut value across all the measured bitstrings (recall that each bitstring corresponds to a cut on our graph). This average cut value is what we will use as the objective function to be passed to the classical optimizer in the next step. We will also set up our simulator backend at this step.

In [ ]:
backend = AerSimulator(method='statevector', device='CPU')

def estimate_circuit_expectation(params, shots=1024):
    n_layers = len(params)//2
    alphas = params[:n_layers]
    betas = params[n_layers:]

    circ = construct_circuit(alphas, betas)
    counts = backend.run(circ, shots=shots).result().get_counts()

    expectation = 0
    for bitstring in counts.keys():
        cut_value = get_cut_value(bitstring)
        expectation += cut_value*counts[bitstring]
    return expectation/shots

The only thing left to do now is to choose an optimizer and find $\alpha$ and $\beta$ parameters which maximize the expected cut value output from our circuit. 

In [ ]:
n_layers = 6
initial_params = np.random.uniform(-.001, .001, 2*n_layers)
opt_res = minimize(estimate_circuit_expectation, initial_params, method='COBYLA')
opt_res

Once this optimization step is finished, we simply run our circuit one final time using the optimized parameters, and the highest value measured cut is taken as the output of our QAOA routine.

For small graph sizes, we can check this QAOA solution against the exact solution found by naively testing all possible cuts.

In [ ]:
optimal_params = opt_res.x
alphas = optimal_params[:n_layers]
betas = optimal_params[n_layers:]

circ = construct_circuit(alphas, betas)
counts = backend.run(circ, shots=1024).result().get_counts()

mode_counts = max(counts.values())
best_cut_val = -np.inf
for bitstring in counts.keys():
    current_cut_val = -get_cut_value(bitstring)
    if current_cut_val > best_cut_val:
        best_cut_val = current_cut_val
        best_cut = bitstring
    if counts[bitstring] == mode_counts:
        mode_cut_val = current_cut_val
        mode = bitstring
qaoa_sol = best_cut


true_sol = naive_exact_maxcut(graph)[0]

print('true solution:', true_sol)
print('qaoa solution:', qaoa_sol)
print('qaoa mode:', mode)
print('\ntrue obj fun val:', -get_cut_value(true_sol))
print('qaoa obj fun val:', best_cut_val)
print('qaoa mode obj fun val:', mode_cut_val)

We can also plot the measurement distribution of our final run using the optimized parameters to see estimate the shape of the quantum state our QAOA circuit is preparing.

In [ ]:
plot_histogram(counts)